## Tutorial and exercises: Few-Shot prompting
*This Colab notebook is part of the [AI for media, art, and design](https://github.com/PerttuHamalainen/MediaAI) course of Aalto University*

*To run the code and complete the exercises, you need an OpenAI account*

This notebook demonstrates text generation using Few-Shot prompting and provides exercises to extend notebook. Solutions to the exercises are also provided, but the code is hidden by default.

**What is Few-Shot prompting?**

Few-shot prompting means that one includes *high-quality examples* of desired outputs in the LLM prompt. This can greatly improve the generated results.  

**Finding/writing the few-shot examples is a key skill to practise** in AI-assisted creative writing. Typically, this includes both writing example text yourself, and browsing and/or scraping the Internet for examples.

For example, we can prompt game ideas using the following prompt:

```
A list of experimental indie game ideas:

---

An FPS game where time only moves when the player moves or performs actions. This allows Matrix-style slow-motion gun ballet, and transforms real-time action into a puzzle.

---

A Sokoban-style game where the player pushes around blocks that are variables, operators, and definitions of a programming language. Blocks that connect to each other form statements that allow the player to define and alter the game's rules. For example, the player can connect "floor", "is", and "lava" to make the floor deadly.

---

```

Above, the few-shot examples describe the indie games Superhot and Baba Is You in a way that makes the core mechanics clear.

Note that this prompt is for LLMs that continue prompt (e.g., OpenAI's davinci-002 and gpt-3.5-turbo-instruct). For chat-based models like ChatGPT, you should change the first line to "Please continue this list of experimental indie game ideas:"


**A few important things:**
* When prompting lists such as game ideas, LLMs cannot distinguish between the prompt and previously generated items. In effect, *previously generated items become new few-shot examples.* This means that even one bad generation can throw the LLM off the rails and quality generally decreases with each item due to small random errors accumulating.
* To prevent the above, it's better to prompt each item separately, making sure that the LLM only sees the original examples. Here, a bit of Python automation can greatly reduce the tedium, as demonstrated in this notebook.
* Generated text quality typically improves with more examples, but going beyond 10 examples tends to provide diminishing gains.
* The ability to generalize from the few-shot examples, a.k.a. *in-context learning*, only emerges in sufficiently large models and was demonstrated for the first time in OpenAI's largest GPT-3 variant (175 billion parameters).
* If you cannot get good quality results and you can produce at least a few hundred examples, an alternative to few-shot prompting is finetuning an LLM, i.e., continuing to train some base LLM with your examples.

**Learning goals:**

* Everyone: Practice producing the few-shot examples and learn to anticipate how different examples change the results
* Those with at least some programming skills: Advanced few-shot prompting techniques such as combinatorial prompting and automatic filtering of the generated results.

**How to use:**
1. Select "Save a copy in drive" from the File menu to make a copy that you can edit in the exercises
2. Select "Run all" from the Runtime menu. In the notebook cell below, enter your OpenAI API key when requested. If you don't have a key, follow [OpenAI's instructions](https://help.openai.com/en/articles/4936850-where-do-i-find-my-openai-api-key)
3. Proceed through the notebook, following the exercise instructions.

**New to Colab notebooks?**

Colab notebooks are browser-based learning environments consisting of *cells* that include either text or code. The code is executed in a Google virtual machine instead of your own computer. You can run code cell-by-cell, and selecting "Run all" as instructed above is usually the first step to verify that everything works. For more info, see Google's [Intro video](https://www.youtube.com/watch?v=inN8seMm7UI) and [curated example notebooks](https://colab.google/notebooks/)


In [1]:
# @title Import libraries and connect to OpenAI
!pip install openai
from openai import OpenAI
from getpass import getpass
OPENAI_API_KEY = getpass('Enter OpenAI API key: ')
client=OpenAI(api_key=OPENAI_API_KEY)

Enter OpenAI API key: ··········


### Basic few-shot prompting of game ideas
The code cell below tries to prompt multiple ideas in one go.

The code is hidden by default to make the interface cleaner. You can show it by clicking on the "Show code".

**Exercise:**
1. Run the code by clicking on the triangle. Repeat this a few times to see how the results are different every time.
2. Change the model to gpt-3.5-turbo-instruct using the drop-down menu. This is the same model as ChatGPT, but for continuing text instead of chat. How do the results change?

**What you should observe:**
* Especially with davinci-002, quality decreases with each idea, as  generated ideas become new few-shot examples for the next ideas and random errors gradually accumulate and throw the model off the rails.
* gpt-3.5-turbo-instruct obeys instructions better than davinci-002, but the generated ideas are more repetitive and generic.

**Why is davinci-002 less reliable but more diverse?**

The davinci-002 is the so-called "base" OpenAI model trained on very large and diverse data. gpt-3.5-turbo-instruct has been finetuned to follow instructions, which tends to increase generation quality but reduce diversity and produce the recognizably boring voice of ChatGPT. Finetuning is typically formulated as an optimization problem for which the optimal solution is to always give the single best answer to a particular request or question. The finetuning datasets are also smaller, which inevitably limits diversity.

In [2]:
model="davinci-002" # @param ["davinci-002", "gpt-3.5-turbo-instruct"] {allow-input: false}

#Define the prompt
prompt="""A list of experimental indie game ideas:

---

An FPS game where time only moves when the player moves or performs actions. This allows Matrix-style slow-motion gun ballet, and transforms real-time action into a puzzle.

---

A Sokoban-style game where the player pushes around blocks that are variables, operators, and definitions of a programming language. Blocks that connect to each other form statements that allow the player to define and alter the game's rules. For example, the player can connect "floor", "is", and "lava" to make the floor deadly.

---

"""

#Query OpenAI completions API
response = client.completions.create(
  model=model,
  prompt=prompt,
  max_tokens=500,
)

#Print the response
print(response.choices[0].text)


A fighting game that's an evolution of Pong. 2 opponents stand on opposite sides of a stage, pushing paddle-like objects at each other to push a ball back and forth. As when playing Pong, the player whose object isn't hit has control of the jump button. The player who wins gets to choose the opponent's control scheme for the next round. Alternately, the opponent who lost gets to choose what objects are used as paddles.

If the player who is denied wins a certain number of games in a row, they win the match.

---

Newtonian physics in reverse. You are a human protagonist going about your daily life trying to work up the courage to give your crush a Valentine's Day card, while the world around you devolves into a chaotic vortex of overgrown weeds, houses rotating around pointy mountains, and highway interchanges drifting in and out of existence.

---

A game that reads from the player's clipboard instead of from a traditional control scheme. In the game world, the player has hacked into 

### Advanced few-shot prompting
The code cell below prompts each idea separately, extracting only the first idea from each generated result. This prevents the LLM from using previously generated ideas as few-shot examples.

The code is hidden by default to make the interface cleaner. You can show it by clicking on the "Show code".

**Exercise:**

1. Run the code by clicking on the triangle. Repeat this a few times to see how the results are different every time.
2. Change the examples to describe your own favorite games. If games are not your thing, skip this step.
3. Change the prompt_start and examples to generate something else such as story ideas, book opening sentences, or book quotes. If you have difficulty articulating the examples, try googling for best book opening sentences or Amazon Kindle's most highlighted passages.



In [3]:
#parameters for the user
model="davinci-002" # @param ["davinci-002", "gpt-3.5-turbo-instruct"] {allow-input: false}
prompt_start="A list of experimental indie game ideas:" # @param {type:"string"}
example1="An FPS game where time only moves when the player moves or performs actions. This allows Matrix-style slow-motion gun ballet, and transforms real-time action into a puzzle." # @param {type:"string"}
example2="A Sokoban-style game where the player pushes around blocks that are variables, operators, and definitions of a programming language. Blocks that connect to each other form statements that allow the player to define and alter the game's rules. For example, the player can connect \"floor\", \"is\", and \"lava\" to make the floor deadly." # @param {type:"string"}
example3="An FPS puzzle game with a \"portal gun\" that can create portals between two flat planes. For example, the player can create one portal on the floor and the second on the ceiling and push an object so that it falls down the floor portal and drops from the ceiling portal on top of some target." # @param {type:"string"}
n_generated=2 #@param {type:"slider", min:1, max:10, step:1}
max_tokens=200 #@param {type:"slider", min:50, max:500, step:10}
examples=[example1,example2,example3] #for convenience, convert the separate examples to a list

#define what separates each list item in the prompt
list_delimiter="---"

#Helper function: Construct a list prompting prompt based on an initial start string and examples
def construct_prompt(prompt_start,examples):
  prompt=prompt_start
  for example in examples:
    if len(example)>0 and (not str.isspace(example)):
      prompt+="\n\n"+list_delimiter+"\n\n"+example
  prompt+="\n\n"+list_delimiter+"\n\n" #add one more delimiter so that the LLM will then add the next list item
  return prompt

#Helper function: Basic generation
def generate(model,prompt,max_tokens):
  response = client.completions.create(
    model=model,
    prompt=prompt,
    max_tokens=max_tokens,
  )
  return response.choices[0].text

#Helper function: Generate a new list item. If multiple items were generated, we only return the first one
def generate_item(model,prompt,max_tokens):
  text=generate(model,prompt,max_tokens)
  text=str.split(text,list_delimiter)[0]
  text=str.strip(text)
  return text

#construct the prompt
prompt=construct_prompt(prompt_start,examples)
print("Prompt:\n")
print(prompt)

#use the prompt to generate new list items
print("Generated list items:")
for i in range(n_generated):
  print("\n\n")
  print(generate_item(prompt=prompt,
                 model=model,
                 max_tokens=200))


Prompt:

A list of experimental indie game ideas:

---

An FPS game where time only moves when the player moves or performs actions. This allows Matrix-style slow-motion gun ballet, and transforms real-time action into a puzzle.

---

A Sokoban-style game where the player pushes around blocks that are variables, operators, and definitions of a programming language. Blocks that connect to each other form statements that allow the player to define and alter the game's rules. For example, the player can connect "floor", "is", and "lava" to make the floor deadly.

---

An FPS puzzle game with a "portal gun" that can create portals between two flat planes. For example, the player can create one portal on the floor and the second on the ceiling and push an object so that it falls down the floor portal and drops from the ceiling portal on top of some target.

---


Generated list items:



Detectives have a codebook that lists words that represent words in a criminal's talk. The words are ide

### Data-conditioned few-shot prompting
Here, we implement a common pattern: ```generated = f(data,few-shot examples)```. In other words, we repeat a generation task multiple times, each time using the same few-shot examples but different data.

Example use cases:

* Prompt game ideas by systematically combining genres and mechanics. Here, each generated item is a game idea, and the item-specific data is a combination such as "FPS + time manipulation" or "Puzzle + rule manipulation". This can increase the diversity of generated ideas, especially when using a finetuned model that produces high quality but less diverse generations.
* Autofilling of a spreadsheet column based on a data column and few-shot examples. For instance, one column might contain Reddit comments about a game or movie and the goal would be to summarize or categorize each comment to understand what people like or dislike.

**Exercise:**

Click to show the code. Instead of game ideas, modify the prompt start, examples, and data to prompt spell names for a game where each spell combines different elements.


In [4]:
# @title
#parameters
model="gpt-3.5-turbo-instruct" # either "davinci-002" or "gpt-3.5-turbo-instruct"
max_tokens=200 #max tokens per generated idea
max_generations=5 #safety: The total number of combinations can grow large, eating up your OpenAI token quota. This is why we limit the number of generated combinations

#This defines the prompt start
prompt_start="A list of experimental indie game ideas based on a genre and mechanic combination:"

#This defines the combined elements. Each element category is a Python list
#inside the []. If you want to combine more than 2 categories, just add one
#or more lists. Note that the total number of generated items is
combined_items=[
    ["FPS","Puzzle","Dating sim","Platformer"], #genres
    ["Time manipulation","Rule manipulation","Camera manipulation"] #mechanics
]

#This defines the few-shot examples.
examples=[
    "FPS + Time manipulation: Time only moves when the player moves or performs actions. This allows Matrix-style slow-motion gun ballet, and transforms real-time action into a puzzle.",
    "Platformer + Camera manipulation: The player moves in a 2D plane inside a 3D world and can jump on any object intersecting the plane. The plane orientation can be changed by changing the orthographic 3D camera.",
    "Puzzle + Rule manipulation: A Sokoban-style game where the player pushes around blocks that are variables, operators, and definitions of a programming language. Blocks that connect to each other form statements that allow the player to define and alter the game's rules. For example, the player can connect \"floor\", \"is\", and \"lava\" to make the floor deadly."
]


#define what separates each list item in the prompt
list_delimiter="---"

#Helper function: Construct the prompt
def construct_prompt(prompt_start,examples):
  prompt=prompt_start
  for example in examples:
    if len(example)>0 and (not str.isspace(example)):
      prompt+="\n\n"+list_delimiter+"\n\n"+example
  prompt+="\n\n"+list_delimiter+"\n\n"
  return prompt

#Helper function: Generate a new list item. If multiple items were generated, we only return the first one
def generate_item(model,prompt,max_tokens):
  response = client.completions.create(
    model=model,
    prompt=prompt,
    max_tokens=max_tokens,
  )
  text=response.choices[0].text
  text=str.split(text,list_delimiter)[0]
  text=str.rstrip(text)
  return text

#construct the prompt
prompt=construct_prompt(prompt_start,examples)
print("Prompt:\n")
print(prompt)

#use the prompt to generate new list items
print("Generated list items:")
import itertools
combinations = list(itertools.product(*combined_items))

for index,combination in enumerate(combinations):
  combination_string=" + ".join(combination)+":"
  item=generate_item(model=model,
                     prompt=prompt+combination_string,
                     max_tokens=max_tokens)
  print("\n\n")
  print(combination_string+item)
  if index>=max_generations:
    break

Prompt:

A list of experimental indie game ideas based on a genre and mechanic combination:

---

FPS + Time manipulation: Time only moves when the player moves or performs actions. This allows Matrix-style slow-motion gun ballet, and transforms real-time action into a puzzle.

---

Platformer + Camera manipulation: The player moves in a 2D plane inside a 3D world and can jump on any object intersecting the plane. The plane orientation can be changed by changing the orthographic 3D camera.

---

Puzzle + Rule manipulation: A Sokoban-style game where the player pushes around blocks that are variables, operators, and definitions of a programming language. Blocks that connect to each other form statements that allow the player to define and alter the game's rules. For example, the player can connect "floor", "is", and "lava" to make the floor deadly.

---


Generated list items:



FPS + Time manipulation: The player has the ability to rewind time, similar to the game "Braid", but the env

### Visualizing generations using embeddings (Not implemented yet, coming in the next update of this notebook)
Here, we demonstrate how embedding vectors can be used for text data visualization.

**What are embedding vectors?**

Modern language models can be used to map any piece of text into real-valued vectors (arrays of floating point values) that encode the meaning of the text in an abstract "embedding space". Similar texts will produce similar embedding vectors (typically, in terms of cosine similarity).

**Exercise:**

Browse the scatter plot using mouse. When using PCA, how do the ideas in the center differ from the ideas far from the ideas in the center?